# Compiling Yelp Restaurant Dataset

## Overview

This notebook compiles a dataset of restaurants from Yelp business data. Then we will combine this with labor market/economic data from BLS and BEA to analyze what causes small business openings and closings.

## Getting Started
1. Download the required libraries:
    ```bash
    pip install -r requirements.txt
    ```

2. Create the `data/` directory and place the downloaded data files in the appropriate subdirectories as specified in the sources section.
    ```bash
    cd predicting-labor-market-trends
    mkdir data
    mkdir data/yelp
    mkdir data/bea
    mkdir data/bls
    ```

## Sources

If API is not specified, data must be downloaded manually.

- *Yelp Business Dataset*: https://business.yelp.com/data/resources/open-dataset/
- *BLS LAUS*: https://www.bls.gov/lau/tables.htm#cntyaa (Download Labor force data by county, 2023 annual averages (XLSX))
- *BEA CAGDP1*: https://apps.bea.gov/regional/downloadzip.htm (Download CAGDP1 folder under GDP tab)
- *County Shapefiles*: https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_5m.zip

## Directory Structure
```
predicting-labor-market-trends/
    ├── data/
    │   ├── yelp/
    |
    │   ├── bea/
    │   │   └── CAGDP1__ALL_AREAS_2001_2023.csv
    │   └── bls/
    │       └── laucnty23.xlsx
    │   └── cb_2018_us_county_5m/
    └── compile_dataset.ipynb
```

## Data Dictionary
| Column Name               | Description                                      | Source                     |
|---------------------------|--------------------------------------------------|----------------------------|
| FIPS5                    | 5-digit FIPS code for the county                 | BLS                        |

In [23]:
import pandas as pd
import requests

YEAR = 2023

In [24]:
# 1. Loading BLS (Unemployment by County)
dtype = {
    'State FIPS Code': str,
    'County FIPS Code': str,
}

# Load each year's data and concatenate
BLS = pd.read_excel(f'./data/bls/laucnty{str(YEAR)[-2:]}.xlsx', skiprows=1, dtype=dtype)

# Create FIPS5 code - combination of State and County FIPS codes
BLS['FIPS5'] = BLS['State FIPS Code'] + BLS['County FIPS Code']

# Drop unnecessary columns and rows with missing FIPS5
BLS.drop(columns=['LAUS Code', 'County Name/State Abbreviation', 'State FIPS Code', 'County FIPS Code', 'Year'], inplace=True)
BLS.dropna(subset=['FIPS5'], inplace=True)

BLS

,Labor Force,Employed,Unemployed,Unemployment Rate (%),FIPS5
0,27741.0,27150.0,591.0,2.1,01001
1,113078.0,110639.0,2439.0,2.2,01003
2,8660.0,8315.0,345.0,4.0,01005
3,8582.0,8376.0,206.0,2.4,01007
4,26501.0,25963.0,538.0,2.0,01009
...,...,...,...,...,...
3216,18428.0,17344.0,1084.0,5.9,72145
3217,2801.0,2689.0,112.0,4.0,72147
3218,7250.0,6416.0,834.0,11.5,72149
3219,9314.0,8548.0,766.0,8.2,72151


In [25]:
# 2. Loading BEA (Real GDP by County)
dtype = {
    'GeoFIPS': str,
}

# Load BEA data
BEA = pd.read_csv('./data/bea/CAGDP1__ALL_AREAS_2001_2023.csv', dtype=dtype, encoding='latin1', skipfooter=4)
BEA = BEA.rename(columns={'GeoFIPS': 'FIPS5'})

# Remove quotes and whitespace from FIPS5 codes
BEA['FIPS5'] = BEA['FIPS5'].str.replace('"', '').str.strip()

# Drop unnecessary columns 
BEA = BEA.drop(columns=[str(year) for year in range(2001, 2024) if year != YEAR] + ['GeoName', 'LineCode', 'Region', 'TableName', 'LineCode', 'IndustryClassification', 'Description', 'Unit'])

# Melt the DataFrame from wide to long format (initially each year is a separate column)
BEA = BEA.melt(id_vars=['FIPS5'], value_vars=[str(YEAR)], var_name='Year', value_name='GDP')
BEA.drop(columns=['Year'], inplace=True)

# Drop rows with missing FIPS5
BEA.dropna(subset=['FIPS5'], inplace=True)
BEA

/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_92639/523649204.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  BEA = pd.read_csv('./data/bea/CAGDP1__ALL_AREAS_2001_2023.csv', dtype=dtype, encoding='latin1', skipfooter=4)


,FIPS5,GDP
0,00000,22671096000
1,00000,115.597
2,00000,27720709000
3,01000,245354674
4,01000,113.267
...,...,...
9529,97000,125.650
9530,97000,1057159025
9531,98000,4525827912
9532,98000,119.181


In [26]:
# 3. Merge Datasets into master panel
economic_df = pd.merge(BLS, BEA, on=['FIPS5'], how='left')
economic_df

,Labor Force,Employed,Unemployed,Unemployment Rate (%),FIPS5,GDP
0,27741.0,27150.0,591.0,2.1,01001,1945909
1,27741.0,27150.0,591.0,2.1,01001,110.403
2,27741.0,27150.0,591.0,2.1,01001,2452642
3,113078.0,110639.0,2439.0,2.2,01003,9435720
4,113078.0,110639.0,2439.0,2.2,01003,127.811
...,...,...,...,...,...,...
9380,18428.0,17344.0,1084.0,5.9,72145,NaN
9381,2801.0,2689.0,112.0,4.0,72147,NaN
9382,7250.0,6416.0,834.0,11.5,72149,NaN
9383,9314.0,8548.0,766.0,8.2,72151,NaN


In [27]:
print("Compiled dataset shape:", economic_df.shape)
print("Number of unique counties:", economic_df['FIPS5'].nunique())

Compiled dataset shape: (9385, 6)
Number of unique counties: 3221


In [28]:
# 4. Descriptive statistics of relevant columns
vars = ['Labor Force', 'Employed', 'Unemployed', 'Unemployment Rate (%)', 'GDP']
economic_df[vars] = economic_df[vars].apply(pd.to_numeric, errors='coerce')
economic_df[vars].describe(include='all')

,Labor Force,Employed,Unemployed,Unemployment Rate (%),GDP
count,9.385000e+03,9.385000e+03,9385.000000,9385.000000,9.244000e+03
mean,5.288523e+04,5.095182e+04,1933.410442,3.649579,5.285505e+06
std,1.734801e+05,1.663361e+05,7351.499142,1.243862,2.893182e+07
min,1.010000e+02,9.300000e+01,3.000000,1.300000,3.405100e+01
25%,4.630000e+03,4.481000e+03,170.000000,2.900000,1.188237e+02
50%,1.142900e+04,1.098500e+04,411.000000,3.500000,4.578975e+05
75%,3.180300e+04,3.072100e+04,1129.000000,4.200000,1.935246e+06
max,5.055267e+06,4.799021e+06,256246.000000,17.600000,9.619081e+08


In [29]:
print("Missing data proportions:")
economic_df[vars].isna().sum() / len(economic_df)

Missing data proportions:


Labor Force              0.000000
Employed                 0.000000
Unemployed               0.000000
Unemployment Rate (%)    0.000000
GDP                      0.015024
dtype: float64

In [30]:
yelp_business = pd.read_json('./data/Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json', lines=True)
yelp_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [31]:
import geopandas as gpd
yelp_business_gdf = gpd.GeoDataFrame(yelp_business, geometry=gpd.points_from_xy(yelp_business.longitude, yelp_business.latitude))


In [32]:
county_shapefile = gpd.read_file('./data/cb_2018_us_county_5m/cb_2018_us_county_5m.shp')
county_shapefile['FIPS5'] = county_shapefile['STATEFP'] + county_shapefile['COUNTYFP']
county_shapefile['geometry'] = county_shapefile['geometry'].to_crs(epsg=4326)
county_shapefile = county_shapefile[['FIPS5', 'geometry']]
yelp_business_gdf = gpd.sjoin(yelp_business_gdf, county_shapefile, how='left', predicate='within')
yelp_business_gdf = yelp_business_gdf.drop(columns=['index_right'])
yelp_business_gdf

/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_92639/2207304939.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  yelp_business_gdf = gpd.sjoin(yelp_business_gdf, county_shapefile, how='left', predicate='within')


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,geometry,FIPS5
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,POINT (-119.7112 34.42668),06083
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",POINT (-90.3357 38.55113),29189
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",POINT (-110.88045 32.22324),04019
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",POINT (-75.15556 39.95551),42101
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",POINT (-75.47166 40.33818),42091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3...",POINT (-113.49205 53.46842),NaN
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3...",POINT (-86.76692 36.11512),47037
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None,POINT (-86.06509 39.90871),18097
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...",POINT (-89.95056 38.78235),17119


In [33]:
full_dataset = pd.merge(yelp_business_gdf, economic_df, on='FIPS5', how='left')
full_dataset

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes,categories,hours,geometry,FIPS5,Labor Force,Employed,Unemployed,Unemployment Rate (%),GDP
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,POINT (-119.7112 34.42668),06083,220025.0,211137.0,8888.0,4.0,3.199681e+07
1,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,POINT (-119.7112 34.42668),06083,220025.0,211137.0,8888.0,4.0,1.164610e+02
2,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,POINT (-119.7112 34.42668),06083,220025.0,211137.0,8888.0,4.0,3.851246e+07
3,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",POINT (-90.3357 38.55113),29189,518804.0,502910.0,15894.0,3.1,9.184719e+07
4,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",POINT (-90.3357 38.55113),29189,518804.0,502910.0,15894.0,3.1,1.156970e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439591,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,...,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...",POINT (-89.95056 38.78235),17119,134203.0,128629.0,5574.0,4.2,1.006750e+02
439592,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,...,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...",POINT (-89.95056 38.78235),17119,134203.0,128629.0,5574.0,4.2,1.457245e+07
439593,jV_XOycEzSlTx-65W906pg,Sic Ink,238 Apollo Beach Blvd,Apollo beach,FL,33572,27.771002,-82.394910,4.5,9,...,"{'WheelchairAccessible': 'True', 'BusinessAcce...","Beauty & Spas, Permanent Makeup, Piercing, Tattoo","{'Tuesday': '12:0-19:0', 'Wednesday': '12:0-19...",POINT (-82.39491 27.771),12057,801413.0,777140.0,24273.0,3.0,1.157729e+08
439594,jV_XOycEzSlTx-65W906pg,Sic Ink,238 Apollo Beach Blvd,Apollo beach,FL,33572,27.771002,-82.394910,4.5,9,...,"{'WheelchairAccessible': 'True', 'BusinessAcce...","Beauty & Spas, Permanent Makeup, Piercing, Tattoo","{'Tuesday': '12:0-19:0', 'Wednesday': '12:0-19...",POINT (-82.39491 27.771),12057,801413.0,777140.0,24273.0,3.0,1.322690e+02


In [34]:
full_dataset = full_dataset.rename(columns={
    'Labor Force': 'labor_force',
    'Employed': 'employed',
    'Unemployed': 'unemployed',
    'Unemployment Rate (%)': 'unemployment_rate',
    'GDP': 'gdp',
})

In [35]:
full_dataset.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 439596 entries, 0 to 439595
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   business_id        439596 non-null  object  
 1   name               439596 non-null  object  
 2   address            439596 non-null  object  
 3   city               439596 non-null  object  
 4   state              439596 non-null  object  
 5   postal_code        439596 non-null  object  
 6   latitude           439596 non-null  float64 
 7   longitude          439596 non-null  float64 
 8   stars              439596 non-null  float64 
 9   review_count       439596 non-null  int64   
 10  is_open            439596 non-null  int64   
 11  attributes         399602 non-null  object  
 12  categories         439289 non-null  object  
 13  hours              372105 non-null  object  
 14  geometry           439596 non-null  geometry
 15  FIPS5              433875 

In [36]:
full_dataset.to_csv('./data/restaurant_economic_data.csv', index=False)